# Which vectorization?

In [2]:
import mlflow
mlflow.set_tracking_uri("https://dagshub.com/prasu202324/YTintel-extension.mlflow")


/Users/prasoon/Developer/MLOPS/YTintel-extension/venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import dagshub
dagshub.init(repo_owner="prasu202324", repo_name="YTintel-extension", mlflow=True)
mlflow.set_experiment("Exp 2 - BoW vs TfIdf")

Accessing as prasu202324

Initialized MLflow to track repo "prasu202324/YTintel-extension"

Repository prasu202324/YTintel-extension initialized!

2026/02/22 15:18:58 INFO mlflow.tracking.fluent: Experiment with name 'Exp 2 - BoW vs TfIdf' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/f5cbf7ee782740a6b2f01facf15ba64b', creation_time=1771753739102, experiment_id='2', last_update_time=1771753739102, lifecycle_stage='active', name='Exp 2 - BoW vs TfIdf', tags={}, workspace='default'>

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow.sklearn
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd
import os


In [6]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [7]:


# Step 1: Function to run the experiment
def run_experiment(vectorizer_type, ngram_range, vectorizer_max_features, vectorizer_name):
    # Step 2: Vectorization
    if vectorizer_type == "BoW":
        vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)
    else:
        vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)

    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    # Step 4: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"{vectorizer_name}_{ngram_range}_RandomForest")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with {vectorizer_name}, ngram_range={ngram_range}, max_features={vectorizer_max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", vectorizer_type)
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", vectorizer_max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: {vectorizer_name}, {ngram_range}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_{vectorizer_name}_{ngram_range}")

# Step 6: Run experiments for BoW and TF-IDF with different n-grams
ngram_ranges = [(1, 1), (1, 2), (1, 3)]  # unigrams, bigrams, trigrams
max_features = 5000  # Example max feature size

for ngram_range in ngram_ranges:
    # BoW Experiments
    run_experiment("BoW", ngram_range, max_features, vectorizer_name="BoW")

    # TF-IDF Experiments
    run_experiment("TF-IDF", ngram_range, max_features, vectorizer_name="TF-IDF")

2026/02/22 19:59:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/22 20:00:25 WARNING mlflow.sklearn: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization. The recommended safe alternative is the 'skops' format. For more information, see: https://scikit-learn.org/stable/model_persistence.html


🏃 View run BoW_(1, 1)_RandomForest at: https://dagshub.com/prasu202324/YTintel-extension.mlflow/#/experiments/2/runs/f09ee3f7ac8348a8af0f8bcd410898f7
🧪 View experiment at: https://dagshub.com/prasu202324/YTintel-extension.mlflow/#/experiments/2


2026/02/22 20:01:07 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/22 20:01:33 WARNING mlflow.sklearn: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization. The recommended safe alternative is the 'skops' format. For more information, see: https://scikit-learn.org/stable/model_persistence.html


🏃 View run TF-IDF_(1, 1)_RandomForest at: https://dagshub.com/prasu202324/YTintel-extension.mlflow/#/experiments/2/runs/9c1a5d74248643059453035662403673
🧪 View experiment at: https://dagshub.com/prasu202324/YTintel-extension.mlflow/#/experiments/2


2026/02/22 20:02:15 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/22 20:02:41 WARNING mlflow.sklearn: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization. The recommended safe alternative is the 'skops' format. For more information, see: https://scikit-learn.org/stable/model_persistence.html


...

🏃 View run TF-IDF_(1, 2)_RandomForest at: https://dagshub.com/prasu202324/YTintel-extension.mlflow/#/experiments/2/runs/8f6faa27e113493c8e6e814d8a246bb7
🧪 View experiment at: https://dagshub.com/prasu202324/YTintel-extension.mlflow/#/experiments/2


2026/02/22 20:04:31 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/22 20:04:57 WARNING mlflow.sklearn: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization. The recommended safe alternative is the 'skops' format. For more information, see: https://scikit-learn.org/stable/model_persistence.html


🏃 View run BoW_(1, 3)_RandomForest at: https://dagshub.com/prasu202324/YTintel-extension.mlflow/#/experiments/2/runs/c4eaa0f3874140afbb8fb014b867ecad
🧪 View experiment at: https://dagshub.com/prasu202324/YTintel-extension.mlflow/#/experiments/2


2026/02/22 20:05:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/22 20:06:05 WARNING mlflow.sklearn: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization. The recommended safe alternative is the 'skops' format. For more information, see: https://scikit-learn.org/stable/model_persistence.html


🏃 View run TF-IDF_(1, 3)_RandomForest at: https://dagshub.com/prasu202324/YTintel-extension.mlflow/#/experiments/2/runs/fad81b68a8404a3cb9f389769fdedc54
🧪 View experiment at: https://dagshub.com/prasu202324/YTintel-extension.mlflow/#/experiments/2
